In [98]:
import pickle
import numpy as np
import math
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score,ConfusionMatrixDisplay
import seaborn as sns
import random
from shapely.geometry import Point, MultiPoint
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import colorsys
from skimage.segmentation import slic
from skimage.data import astronaut
from skimage.color import label2rgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import cv2 as cv
from google.colab.patches import cv2_imshow
import random


In [99]:
def load_dataset():
  image_pickle_file_path = '/content/drive/MyDrive/project/images.pkl'
  label_pickle_file_path = '/content/drive/MyDrive/project/label.pkl'

  with open(image_pickle_file_path, 'rb') as file:
    images = pickle.load(file)

  with open(label_pickle_file_path, 'rb') as file:
    labels = pickle.load(file)

  images = images.reshape(images.shape[0], -1)

  return images, labels

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
def classify(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy * 100:.2f}%")
  return accuracy

In [ ]:
images, true_labels = load_dataset()
images[0]

In [ ]:

def convertToHSV(image):

  pixels = np.array(image)
  rgb_image = pixels.reshape((499, 499,3))
  r, g, b = rgb_image[:, :, 0], rgb_image[:, :, 1], rgb_image[:, :, 2]
  vectorized_rgb_to_hsv = np.vectorize(colorsys.rgb_to_hsv)
  hsv_image = vectorized_rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)
  hsv_image = np.transpose(hsv_image, (1, 2, 0))
  return hsv_image


newimg=convertToHSV(images[33])
plt.imshow(newimg, cmap='hsv')
plt.axis('off')
plt.show()

In [104]:


def convertToPolar(i,j):
    center=249
    dx = i - center
    dy = j - center
    distance = math.sqrt(dx**2 + dy**2)
    angle_rad = math.atan2(dy, dx)
    angle_degrees = math.degrees(angle_rad)
    return distance, angle_degrees

In [106]:


def create(image):
    pixels={}
    pixels['info'] = []
    pixels['x']=[]
    pixels['y']=[]
    scaler = MinMaxScaler()

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image[i, j]
            distance, angle = convertToPolar(i, j)
            normalized_pixel = [pixel[0], pixel[1], pixel[2] ,distance, angle]
            pixels['info'].append(normalized_pixel)
            pixels['x'].append(j)
            pixels['y'].append(500-i)

    pixels['info'] = np.array(pixels['info'])
    pixels['info'] = scaler.fit_transform(pixels['info'])*np.array([18,6,6,1,1])
    return pixels

In [107]:
data = create(newimg)
# kmeans = KMeans(n_clusters=4, n_init=10, random_state=42).fit(data['info'])
# labels = kmeans.labels_

# plt.scatter(data['x'], data['y'], c=labels, cmap='viridis')
# plt.title('K-means Clustering')
# plt.show()

def use_kmeans_cv2(img):
  np.random.seed(42)
  Z = img.reshape((-1, 3)).astype(np.float32)
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  K = 4
  ret, labelss, center = cv.kmeans(Z, K, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)
  center = np.uint8(center)
  res = center[labelss.flatten()]
  res2 = res.reshape((499,499,3))
  # cv2_imshow(res2)
  return  center ,labelss.flatten()

In [108]:
def groupByLabel(data,labels):
  group={}
  for info,x,y,label in zip(data['info'],data['x'],data['y'],labels):
    if(label not in group.keys()):

      group[label]={}
      group[label]['x']=[]
      group[label]['y']=[]
      group[label]['info']=[]

    group[label]['info'].append(info)
    group[label]['x'].append(x)
    group[label]['y'].append(y)
  return group

def colorVariance(img,group):
  total_variance=0
  h_list=[]
  s_list=[]
  v_list=[]
  for x,y in zip(group['x'],group['y']):
    color= img[500-y][x]
    h_list.append(color[0])
    s_list.append(color[1])
    v_list.append(color[2])
  h_variance = np.var(h_list)
  s_variance = np.var(s_list)
  v_variance = np.var(v_list)

  return h_variance , s_variance , v_variance

def colorAverage(img,group):
  total_variance=0
  h_list=[]
  s_list=[]
  v_list=[]
  for x,y in zip(group['x'],group['y']):
    color= img[500-y][x]
    h_list.append(color[0])
    s_list.append(color[1])
    v_list.append(color[2])
  h_average = np.average(h_list)
  s_average = np.average(s_list)
  v_average = np.average(v_list)

  return h_average , s_average , v_average

In [109]:
def hAverage(img,group):
  total_variance=0
  h_list=[]
  for x,y in zip(group['x'],group['y']):
    color= img[500-y][x]
    h_list.append(color[0])
  h_variance = np.average(h_list)
  h_min=np.amin(h_list)
  h_max=np.amax(h_list)
  return h_variance,h_min,h_max

In [110]:
def pointVariance(group):
  dist=[]
  ang=[]
  for i in group['info']:
    dist.append(i[3])
    ang.append(i[4])

  return np.var(dist),np.var(ang)

def pointAverage(group):
  dist=[]
  ang=[]
  for i in group['info']:
    dist.append(i[3])
    ang.append(i[4])

  return np.average(dist),np.average(ang)

In [111]:
#محیط

def calculaterPerimeter(group):
  point_coordinates = [Point(x, y) for x,y in zip(group['x'],group['y']) ]
  multi_point = MultiPoint(point_coordinates)
  convex_hull = multi_point.convex_hull

  bounding_box = multi_point.bounds
  return convex_hull.length

In [112]:

def circularity(group):
  primeter=calculaterPerimeter(group)
  area=len(group['x'])
  return 4*np.pi*area/(primeter**2)

In [113]:
def aspect_ratio(group):
  ymax=np.amax(group['x'])
  ymin=np.amin(group['x'])
  xmax=np.amax(group['y'])
  xmin=np.amin(group['y'])
  ydif=ymax-ymin
  xdif=xmax-xmin
  return (ydif /xdif)

In [114]:
scaler = MinMaxScaler()
# c_list=[]
# labels_list=[]
c_list=np.load('/content/drive/MyDrive/project/c_list.npy')
labels_list=np.load('/content/drive/MyDrive/project/labels_list.npy')
def findFeatures(images):
  features=[]
  vectors=[]
  i=0
  for image in images:
    imgvectors=[]
    hsvImg=convertToHSV(image)
    data = create(hsvImg)
    centers  , labelss =c_list[i] ,labels_list[i]  #use_kmeans_cv2(image)
    # c_list.append(centers)
    # labels_list.append(labelss)


    unique, counts = np.unique(labelss, return_counts=True)

    groups=groupByLabel(data,labelss)
    for g,c in zip(unique,counts):
      groups[g]['size']=c



    for v in groups.values():
        dist,ang=pointAverage(v)
        h_avg,s_avg,v_avg=colorAverage(hsvImg,v)
        var_dist,var_ang= pointVariance(v)
        h_variance , s_variance , v_variance = colorVariance(hsvImg,v)
        vector=[h_avg,s_avg,v_avg,dist,ang,circularity(v),aspect_ratio(v),calculaterPerimeter(v),var_dist,var_ang,h_variance , s_variance , v_variance ]
        features.append(vector)

    i=i+1
    print(f'{int(i)}/{len(images)}')

  return features

def scale(data,coefficients):
  features=scaler.fit_transform(data)*np.array(coefficients)
  vectors=features.reshape(-1,4,13)
  return features,vectors

In [115]:
def getRandomSamples(count):
  random.seed(44)
  indexes = random.sample(range(len(images)), count)
  random_images = [images[i] for i in indexes]
  random_labels = [true_labels[i] for i in indexes]
  return random_images,random_labels

def getFirstSamples(count):
  return images[:count],true_labels[:count]

In [116]:
my_array,label_array= getRandomSamples(560);
# my_array,label_array= getFirstSamples(100);
feature_list=np.load('/content/drive/MyDrive/project/feature_list.npy')
# feature_list=findFeatures(my_array)
# np.save('/content/drive/MyDrive/project/feature_list',feature_list)
# np.save('/content/drive/MyDrive/project/c_list',c_list)
# np.save('/content/drive/MyDrive/project/labels_list',labels_list)
feat,vect=scale(feature_list,[1]*13)
cluster = KMeans(n_clusters=40, n_init=10, random_state=42).fit(np.array(feat))
labelsDB = cluster.labels_

In [117]:
def groupFeaturesByLabels(data,labels):
  group={}
  vec_to_group={}
  for vec,label in zip(data,labels):
    if(label not in group.keys()):
      group[label]=[]
    group[label].append(vec)
    vec_to_group[f'{vec}']=label

  return group,vec_to_group
gro,vec_to_group=groupFeaturesByLabels(feat,labelsDB)

In [118]:

def meanvect(vect):
  list_mean_vect=[]
  for i in vect:
    mean_vect=np.mean(i,axis=0)
    list_mean_vect.append(mean_vect)
  return list_mean_vect
backup_vect=vect.copy()
backup_feat=feat.copy()

In [ ]:
from urllib.request import parse_http_list
vect=backup_vect.tolist()
vectpop=[]
count=0
count_list=[]
worth_part=[]
feat_list=meanvect(vect)
acc=classify(feat_list,label_array)
clf=RandomForestClassifier(n_estimators=100, random_state=42)
best_area=[[0,1,2,3] for i in range(len(my_array))]
for k in range(3):
  worth_area=[]
  delete_index=0
  clf.fit(feat_list,label_array)
  #for i in reversed(range(len(my_array))):
  for i in range(len(my_array)):
    list_of_all=[]
    worth_part,max_acc = [],0#0,100
    img=my_array[i]
    print(f"i={i}, k={k}")
    for j in range(4-k):
      copy_vect=vect.copy()
      vectpop=copy_vect[i].pop(j)
      #feat_list=meanvect(copy_vect)
      mean_vect=np.mean([vectpop],axis=0)
      pred=clf.predict_proba([mean_vect])[0]
      label=label_array[i]
      acc=pred[label]
      print(f"Accuracy: {acc* 100:.2f}%")
      #classify(feat_list,label_array)
      if (acc >= max_acc):
        max_acc =acc
        worth_part.append(j)
        count=len(worth_part)
      copy_vect[i].insert(j,vectpop)
    if(count>1):
      max_dist=0
      for delete in worth_part:
        if(vect[i][delete][3] >= max_dist):
          max_dist=vect[i][delete][3]
          delete_index=delete
    else:
      delete_index=worth_part[0]
    worth_area.append(delete_index)
    #worth_area.insert(0,delete_index)
  for q in range(len(worth_area)):
    vect[q].pop(worth_area[q])
    best_area[q].pop(worth_area[q])


print("----final----")

feat_list=meanvect(vect)
acc=classify(feat_list,label_array)

In [ ]:
index=127
test_image=my_array[index].reshape((499,499,3))
def get_result(image_index):
  center,la=c_list[image_index],labels_list[image_index]#use_kmeans_cv2(test_image)
  res = center[la]
  res2=res.reshape((499,499,3))
  label_re=la.reshape((499,499))
  color=center[best_area[image_index][0]]
  for i in range(499):
    for j in range(499):
      if(label_re[i][j] in best_area[image_index]):
          res2[i][j]=color
      else:
        res2[i][j]=np.array([0,0,0])
  return res2
fig = plt.figure(figsize=(10, 7))
columns = 2
rows = 2
fig.add_subplot(rows, columns, 1)
plt.imshow(test_image)
plt.title("Image")
plt.axis('off')
fig.add_subplot(rows, columns, 2)
plt.imshow(get_result(index))
plt.title("result")
plt.axis('off')

In [ ]:
def classfiyAndremoveMistakes(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)#label
  y_pred_prob = clf.predict_proba(X_test)#label & %
  mistakes=[]
  trues=[]
  selected_indexes=[]
  for a in X_test:
    i=0;
    for d in datapoints:
      if(np.array_equal(a,d)):
         selected_indexes.append(i)
         break
      i=i+1
  imgs=[my_array[i] for i in selected_indexes]
  labl=[label_array[i] for i in selected_indexes]
  total=len(y_pred)#.shape[0]#len
  # print(total)
  deleted_count=0;
  for i,j,z,h in zip(selected_indexes,labl,y_pred,y_pred_prob):
    if j!=z:
      mistakes.append((i,h[j]))
    else:
      trues.append((i,h[j]))

  sorted_tuples = sorted(mistakes, key = lambda x: x[1])
  mistakess=sorted_tuples[int(0.2*total):]
  print(f'deleted:{mistakess}')
  datapointse=datapoints.copy()
  labelse=labels.copy()
  for i in sorted(mistakess,reverse=True):
    datapointse.pop(i[0])
    labelse.pop(i[0])
  return datapointse,labelse,mistakess, trues

datapoints,labelss,mistakes,trues=classfiyAndremoveMistakes(feat_list,label_array)

In [ ]:
fig = plt.figure(figsize=(5, 40))
columns = 2
rows = len(mistakes)
i=1
for index,acc in mistakes:
  img=my_array[index].reshape((499,499,3))
  fig.add_subplot(rows, columns, i)
  plt.imshow(img)
  if i==1:
    plt.title("Image")
  plt.axis('off')
  fig.add_subplot(rows, columns, i+1)
  plt.imshow(get_result(index))
  if i==1:
    plt.title("result")
  plt.axis('off')
  i=i+2

In [ ]:
fig = plt.figure(figsize=(5, 30))
columns = 2
trues_sample=random.sample(trues,10)
rows = len(trues_sample)
i=1
for index,acc in trues_sample:
  img=my_array[index].reshape((499,499,3))
  fig.add_subplot(rows, columns, i)
  plt.imshow(img)
  plt.title("Image")
  plt.axis('off')
  fig.add_subplot(rows, columns, i+1)
  plt.imshow(get_result(index))
  plt.title(f"probability:{acc}")
  plt.axis('off')
  i=i+2

In [ ]:
coefficients=[0]*13
plt.figure(figsize=(13, 13))
correlation_matrix = np.corrcoef(datapoints, rowvar=False)
sns.heatmap(correlation_matrix, annot=True)
selected_feats=[]
best_acc=0
best_feat=[]
best_coefficients=0
acc_list=[]
flag=False
for i in range(13):
  index_best=0
  coef_lists=[]
  mean_list=[]
  best_score=0
  selected_acc=0
  for index in range(13):
      if (index in selected_feats):
          continue
      coefficients[index]=1
      final_feats=datapoints*np.array(coefficients)
      print(f"selected count: {i} -  checking feature {index} ")
      acc=classify(final_feats,labelss)
      if(best_acc < acc):
        best_acc=acc
        best_feat=selected_feats.copy()
        best_feat.append(index)
        best_coefficients=coefficients.copy()
      coefficients[index]=0
      max_corr=0
      corr=0
      for s in selected_feats:
          corr=abs(correlation_matrix[index,s])
          if ( max_corr > corr):
              max_corr=corr
      normalized_corr=1-max_corr
      score = 2 / ((1 / acc) + (1 / normalized_corr))
      if(best_score < score):
        best_score=score
        index_best=index
        selected_acc=acc

  acc_list.append(selected_acc)
  if(i>1):
    diff_acc= selected_acc - acc_list[i-1]
    if (diff_acc<0):
      if (diff_acc<=-0.03 or flag==True):
        break;
      else:
        flag=True
  print(f'feature {index_best} selected \n')
  selected_feats.append(index_best)
  coefficients[index_best]=1
print(f"selected_feats {selected_feats}")
print(f"acc {best_acc*100:.2f}%")
print(f"best_coefficients{best_coefficients}")


In [ ]:
plt.plot(range(1,len(acc_list)+1), acc_list)

plt.xlabel('iteration')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
feature_name=['h_average','s_average','v_average','distance_average','angle_average','circularity','aspect_ratio','perimeter','distance_variance','angle_variance','h_variance ','s_variance ','v_variance']
print('Ranking of features:')
for i,j in enumerate(selected_feats):
  print(f'{i+1}.{feature_name[j]}')